<div style = "background-color:indigo"><center>
<h1 style="font-size: 50px; font-weight: bold; color:goldenrod; border-top: 3px solid goldenrod; padding-top: 10px">AI California Legislative Policy Analysis (CALPA)</h1>
<div style="font-size: 35px; font-weight: bold; color: goldenrod"> Part 1 - Preliminary Data Processing</div>
<div style="font-size: 30px; font-weight: bold; color: goldenrod; border-bottom: 3px solid goldenrod; padding-bottom: 20px">v.1.0 April 2025</div>
</center></div>

This is the main notebook for the AI California Legislative Policy Analysis (CALPA) project. The goal of this project is to analyze California legislative bills using natural language processing (NLP) techniques. This notebook will cover the preliminary data processing steps, including data loading, cleaning, and preparation for analysis.
The project is divided into several parts, each focusing on a specific aspect of the analysis. The first part will cover the data loading and cleaning process, while subsequent parts will focus on feature extraction, model training, and evaluation.

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">1. Preliminaries</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.1 Referencing Libraries and Initialization</h2>

If needed to reset the kernel, please run the following cell:

In [1]:
#%reset

Instantiating python libraries for the project

In [2]:
# Import required libraries
import os
import time
from datetime import date
from datetime import datetime
import json
import mimetypes
import glob
import base64
import zipfile
import io
import dotenv
import requests
import pandas as pd

Load the local python modules containing classes and functions for the project from the local directory. There are two modules:
- `calpa`: This module contains the main classes and functions for the project.
- `legiscan`: This module contains the classes and functions for the LegiScan API.

In [3]:
# Load the Calpa module located in the scripts/python/calpa directory
from calpa import Calpa, LegiScan

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">1.2. Project and Workspace Variables</h2>

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Load Environment Variables</h3>

Define and maintain project, workspace and metadata. Below we load the environment variables from the `.env` file. The environment variables are used to configure the project and workspace settings. The `dotenv` library is used to load the environment variables from the `.env` file into the Python environment. The environment also contains the LegiScan API key, which is used to access the LegiScan API. The API key is stored in the `LEGISCAN_API_KEY` environment variable. The `dotenv` library is used to load the environment variables from the `.env` file into the Python environment.

In [4]:
# Load environment variables from .env file
dotenv.load_dotenv(os.path.join(os.getcwd(), '.env'))

True

<h3 style="font-weight:bold; color:lime; padding-left: 50px">Main Class Instantiation</h3>

Instantiate the two main classes for the project:
- `calpa`: This class is used to access the CALPA API and retrieve legislative data.
- `legiscan`: This class is used to access the LegiScan API and retrieve legislative data.

In [5]:
# Instantiate the LegiScan and Calpa classes
calpa = Calpa()
legiscan = LegiScan()

Create project metadata for the project

In [6]:
# Create project metadata for the AI project
prjMetadata = calpa.projectMetadata("AI", "0")

Project Global Settings:
- Name: California Legislative Policy Analysis
- Title: AI Legislative Policy Analysis
- Version: 1.0
- Author: Dr. Kostas Alexandridis, GISP
Data Dates
- Start Date: 2010-12-02
- End Date: 2025-04-18
- Periods: 2009-2010, 2011-2012, 2013-2014, 2015-2016, 2017-2018, 2019-2020, 2021-2022, 2023-2024, 2025-2026


Create the project directories dictionary

In [7]:
# Create the project directories dictionary
prjDirs = calpa.projectDirectories(os.getcwd())

Directory Global Settings:

General:
- Project: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA
- Admin: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\admin
- Metadata: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\metadata
- Analysis: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\analysis
Scripts:
- Python Calpa Module: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\calpa
- Markdown Scripts: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\markdown
- RIS Scripts: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\ris
Data:
- Main Data: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data
- Documents: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data\docs
- LegiScan: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data\legis
- LookUp: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data\lookup
- Markdown: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data\md
- RIS: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data\bbl
Graphics:
- Main Graphics: c:\Users\ktale\OneDrive\Documents\GitHub\Ca

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">2. Baseline LegiScan Data</h1>

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.1. Session List</h2>

Using the LegiScan API, we will retrieve the list of sessions for California. This will be used to get the session ID for the current session and the previous session. The session ID is needed to retrieve the bills for each session.

In [8]:
# Get the list of sessions from LegiScan
sessionList = legiscan.getSessionList()

Convert the session list to a pandas dataframe

In [9]:
# Convert the sessionList to a pandas DataFrame
sessionDf = pd.DataFrame(sessionList)
sessionDf.head()

,2025-2026,2023-2024,2021-2022,2019-2020,2017-2018,2015-2016,2013-2014,2011-2012,2009-2010
session_id,2172,2016,1791,1624,1400,1120,993,82,30
state_id,5,5,5,5,5,5,5,5,5
state_abbr,CA,CA,CA,CA,CA,CA,CA,CA,CA
year_start,2025,2023,2021,2019,2017,2015,2013,2011,2009
year_end,2026,2024,2022,2020,2018,2016,2014,2012,2010


We need to compare the session list we obtained from the Legiscan API with the previous session list (stored in the disk under `data/lookup/sessionList.json`). Here, we open the stored session list into a new dictionary called `sessionListStored`.

In [10]:
# Obtain the stored sessions list from JSON dictionary on disk (data/lookup directory)
sessionListStored = legiscan.getStoredSessions()

Now that we have both the session lists (the one from the legiscan api, `sessionList`, and the stored version, `sessionListStored`), we can compare them. We will check if the session list from the LegiScan API is the same as the session list stored in the disk. If they are not the same, we will first identify which sessions need updating, and will later update the stored session list with the new session list from the LegiScan API. We will also check if there are any new sessions that have been added to the LegiScan API since the last time we retrieved the session list.

The function method `matchHash` from the legiscan module class, uses the hash values to compare the two lists. In this case the relevant JSON keys are `sesion_hash` for each `session_id`.

In [11]:
# Compare the sessionList and sessionListStored dictionaries for any changes
unmatchedSessions = legiscan.matchHash(sessionList, sessionListStored, "session_hash", silent=True)

# if the unmatchedSessions is empty, print "All sessions match", and delete the unmatchedSessions variable
if unmatchedSessions is None:
    print("All sessions match")
    del unmatchedSessions

All sessions match


As a final step, export the LegiScan query records to the `data/legis/json` directory as a JSON file for future reference.

In [12]:
# Export the sessionList to a JSON file in the data/legiscan/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "sessionList.json"), "w", encoding="utf-8") as f:
    json.dump(sessionList, f, ensure_ascii=False, indent=4)
del f

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.2. Session People</h2>

In this step, we will obtain the list of California legislature members (Senate and Assembly) for each of the legislative sessions. This will be used to get the list of members for each session. The session ID is needed to retrieve the members for each session. We will use the LegiScan API to retrieve the list of members for each session.

The `legiscan.getSessionPeople` method retrieves the list of members for each session. The session ID is passed as an argument to the method. The method returns a list of members for each session. The list of members is stored in a dictionary called `sessionPeople`. The dictionary contains the session ID as the key and the list of members as the value.

In [13]:
# Get the list of session people from LegiScan
sessionPeople = {}
for key, value in sessionList.items():
    sessionId = value["session_id"]
    sessionPeople[key] = legiscan.getSessionPeople(sessionId)
del key, value, sessionId

Similarly with the legislative session list, we will compare the session people list we obtained from the LegiScan API with the previous session people list (stored in the disk under `data/lookup/sessionPeople.json`). Here, we open the stored session people list into a new dictionary called `sessionPeopleStored`.

In [14]:
# Obtain the stored session People list from JSON dictionary on disk (data/lookup directory)
sessionPeopleStored = legiscan.getStoredPeople()

This time, the task is not that simple, since `sessionPeople` lists are nested for each session. The comparison of the Legislature members needs to be done in a loop for each session. In the following code segment, we perform this task in sequential steps:

1. Create a dictionary named `unmatchedPeople` to hold the unmatched session people (will be nested for each session).
2. Loop through the `sessionPeople` and `sessionPeopleStored` dictionaries to compare the session people lists for each session.
3. For each session, compare the session people lists and store the unmatched session people in the `unmatchedSessionPeople` dictionary, based on the `person_hash` key attribute on both lists. We will use the `matchHash` method from the `legiscan` module to compare the two lists.
4. If there are any unmatched session people, we will update the `unmatchedPeople` dictionary with the unmatched session people (for each session).
5. Finally, we will check if there are any unmatched session people in the `unmatchedPeople` dictionary. If there are, we will update the `sessionPeopleStored` dictionary with the unmatched session people and save it to the disk. 

In [15]:
# Compare the sessionPeople and sessionPeopleStored dictionaries for any changes
# Create a dictionary to store unmatched people
unmatchedPeople = {}
# Iterate through each session and compare the people lists
for key, value in sessionPeople.items():
    unmatchedPeople[key] = {}
    unmatched = legiscan.matchHash(sessionPeople[key]["people"], sessionPeopleStored[key]["people"], "person_hash", silent=True)
    # If there are unmatched people, store them in the unmatchedPeople dictionary
    unmatchedPeople[key] = unmatched if unmatched is not None else None
del key, value, unmatched

# if the unmatchedPeople is empty, print "All people match", and delete the unmatchedPeople variable
if all(not value for value in unmatchedPeople.values()):
    print("All people match")
    # Delete the unmatchedPeople variable
    del unmatchedPeople

All people match


As a final step, export the LegiScan query data for the session people to the `data/legis/json' directory for future reference.

In [16]:
# Export the sessionPeople to a JSON file in the data/legiscan/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "sessionPeople.json"), "w", encoding="utf-8") as f:
    json.dump(sessionPeople, f, ensure_ascii=False, indent=4)
del f

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.3. Dataset List</h2>

In this session we will obtain the list of datasets and their attributes for each of the California Legislative Session from LegiScan. This process is needed to obtain the dataset `access_key` for each session, and consequently to use it in query dataset data in a further step.

The `legiscan.getDatasetList` method retrieves the list of datasets for each session. The session ID is passed as an argument to the method. The method returns a list of datasets for each session. The list of datasets is stored in a dictionary called `datasetList`. The dictionary contains the session ID as the key and the list of datasets as the value.

In [18]:
# Get the list of datasets from LegiScan for each legislative session
datasetList = legiscan.getDatasetList()

Obtain the stored dataset list from the disk. The list will be used to compare with the LegiScan API dataset list. The stored dataset list is stored in the `data/lookup/datasetList.json` file. The dataset list contains the list of datasets for each session. The dataset ID is needed to retrieve the datasets for each session. We will use the LegiScan API to retrieve the list of datasets for each session. Here, we open the stored dataset list into a new dictionary called `datasetListStored`.

In [ ]:
# Obtain the stored dataset list from JSON dictionary on disk (data/lookup directory)
datasetListStored = legiscan.getStoredDatasetList()

Now that we have both the dataset lists (the one from the legiscan api, `datasetList`, and the stored version, `datasetListStored`), we can compare them. We will check if the dataset list from the LegiScan API is the same as the dataset list stored in the disk. If they are not the same, we will first identify which datasets need updating, and will later update the stored dataset list with the new dataset list from the LegiScan API. We will also check if there are any new datasets that have been added to the LegiScan API since the last time we retrieved the session list.

The function method `matchHash` from the legiscan module class, uses the hash values to compare the two lists. In this case the relevant JSON keys are `dataset_hash` for each `session_id`.

In [19]:
# Compare the datasetList and datasetListStored dictionaries for any changes
unmatchedDatasets = legiscan.matchHash(datasetList, datasetListStored, "dataset_hash", silent=True)

# if the unmatchedSessions is empty, print "All sessions match", and delete the unmatchedSessions variable
if unmatchedDatasets is None:
    print("All datasets match")
    del unmatchedDatasets

All datasets match


As a final step, export the LegiScan query records to the `data/legis/json` directory as a JSON file for future reference.

In [20]:
# export the datasetList to a JSON file in the data/legis/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "datasetList.json"), "w", encoding="utf-8") as f:
    json.dump(datasetList, f, ensure_ascii=False, indent=4)
del f

<h2 style="font-weight:bold; color:dodgerblue; border-bottom: 1px solid dodgerblue; padding-left: 25px">2.4. Master List</h2>

This step is to obtain the master list of datasets for each session. The master list contains the list of bills for each legislative session. The master list is used to get the list of bills for each session. The session ID is needed to retrieve the bills for each session. We will use the LegiScan API to retrieve the list of bills for each session.

There are two options for this method. The first obtains the master list with bill attributes (when `raw = False`), and the second obtains the raw master list containing only the bill_ID and hash (when `raw = True`)

We will use the `legiscan.getMasterList(sessionID, raw)` method and will store the results in a dictionary called `masterList` or `masterListRaw` depending on the option provided in the method invocation.

In [43]:
# Get the Raw Master List from LegiScan for each legislative session
masterListRaw = {}
for key, value in sessionList.items():
    sessionId = value["session_id"]
    masterListRaw[key] = legiscan.getMasterList(sessionId, raw = True)
del key, value, sessionId

In [44]:
# Get the Master List from LegiScan for each legislative session
masterList = {}
for key, value in sessionList.items():
    sessionId = value["session_id"]
    masterList[key] = legiscan.getMasterList(sessionId, raw = False)
del key, value, sessionId

Obtain the stored master lists (both the raw and full) from the disk. The lists will be used to compare each bill with the LegiScan API master list. The stored dataset list is stored in the `data/lookup/mastertList.json` or `data/lookup/masterListRaw.json` files. The master lists contain the list of bills for each session. The dataset ID is needed to retrieve the bills for each session. We will use the LegiScan API to retrieve the list of datasets for each session. Here, we open the stored dataset list into a new dictionary called `masterListStored` and `masterListRawStored`.

In [47]:
# Get the stored raw master list from JSON dictionary on disk (data/lookup directory)
masterListRawStored = legiscan.getStoredMasterList(raw = True)
# Get the stored master list from JSON dictionary on disk (data/lookup directory)
masterListStored = legiscan.getStoredMasterList(raw = False)

Now that we have both the master lists (the one from the legiscan api, `masterListRaw`, and the stored version, `masterListRawStored`), we can compare them. We will check if the master list from the LegiScan API is the same as the master list stored in the disk. If they are not the same, we will first identify which bills need updating, and will later update the stored master list with the new master list from the LegiScan API. We will also check if there are any new bills that have been added to the LegiScan API since the last time we retrieved the session list.

The function method `matchHash` from the legiscan module class, uses the hash values to compare the two lists. In this case the relevant JSON keys are `chanbe_hash` for each `session_id`.

In [50]:
# Compare the masterList and masterListStored dictionaries for any changes
# Create a dictionary to store unmatched bills
unmatchedMasterList = {}
for key, value in masterListRaw.items():
    # if key is not "session"
    if key != "session":
        unmatchedMasterList[key] = {}
        unmatched = legiscan.matchHash(masterListRaw[key]["bills"], masterListRaw[key]["bills"], "change_hash", silent=True)
        # If there are unmatched bills, store them in the unmatchedMasterList dictionary
        unmatchedMasterList[key] = unmatched if unmatched is not None else None
del key, value, unmatched

# if the unmatchedMasterList is empty, print "All bills match", and delete the unmatchedMasterList variable
if all(not value for value in unmatchedMasterList.values()):
    print("All bills match")
    # Delete the unmatchedMasterList variable
    del unmatchedMasterList

All bills match


As a final step, export both the LegiScan query records (raw and full master list) to the `data/legis/json` directory as a JSON file for future reference.

In [45]:
# export the raw master list to a JSON file in the data/legis/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "masterListRaw.json"), "w", encoding="utf-8") as f:
    json.dump(masterListRaw, f, ensure_ascii=False, indent=4)
del f

In [46]:
# export the master list to a JSON file in the data/legis/json directory
with open(os.path.join(prjDirs["pathDataLegis"], "json", "masterList.json"), "w", encoding="utf-8") as f:
    json.dump(masterList, f, ensure_ascii=False, indent=4)
del f

<div style = "background-color:indigo"><center>
<h1 style="font-weight:bold; color:goldenrod; border-top: 2px solid goldenrod; border-bottom: 2px solid goldenrod; padding-top: 5px; padding-bottom: 10px">End of Script</h1>
</center></div>